In [ ]:
import uproot
import numpy as np
import pkgutil
import uproot_methods
import matplotlib as mpl
import matplotlib.pyplot as plt
import max_functions as mf
%matplotlib inline

#to smooth the MFPADs
import scipy as sp
import scipy.ndimage

import mplhep as hep
plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS
import seaborn as sns
from scipy.interpolate import griddata

#to see which classes have been defined so far
[modname for importer, modname, ispkg in pkgutil.walk_packages(uproot_methods.classes.__path__)]

path = "angular_distr_el/ALL_CH/test/MFPADs_e_multinew_check/MFPAD3D_ALLdir_usr_gated"

In [ ]:
#low binning
# fileRCR = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\R-C3H3F3O_550eV_CR_newEfield_ALLCH_MFPAD_30_t0.root")
# # fileRCR["angular_distr_el/ALL_CH/test/MFPADs_e_multinew_check"].keys()
# # dict(fileRCR["angular_distr_el/ALL_CH/test/MFPADs_e_multinew_check"].classes())
# fileSCR = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\S-C3H3F3O_550eV_CR_newEfield_ALLCH_MFPAD_30_t0.root")
# fileRCL = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\R-C3H3F3O_550eV_CL_newEfield_ALLCH_MFPAD_30_t0.root")
# fileSCL = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\S-C3H3F3O_550eV_CL_newEfield_ALLCH_MFPAD_30_t0.root")

In [ ]:
#high binning
fileRCR = uproot.open(r"C:\Users\Giammarco\Desktop\PYTHON_graphs\DATA\Experiments\R-C3H3F3O_550eV_CR_newEfield_ALLCH_MFPAD_30_t0_higherbins.root")
fileSCR = uproot.open(r"C:\Users\Giammarco\Desktop\PYTHON_graphs\DATA\Experiments\S-C3H3F3O_550eV_CR_newEfield_ALLCH_MFPAD_30_t0_higherbins.root")
fileRCL = uproot.open(r"C:\Users\Giammarco\Desktop\PYTHON_graphs\DATA\Experiments\R-C3H3F3O_550eV_CL_newEfield_ALLCH_MFPAD_30_t0_higherbins.root")
fileSCL = uproot.open(r"C:\Users\Giammarco\Desktop\PYTHON_graphs\DATA\Experiments\S-C3H3F3O_550eV_CL_newEfield_ALLCH_MFPAD_30_t0_higherbins.root")

#low fixed binning
fileRCR = uproot.open(r"C:\Users\Giammarco\Desktop\PYTHON_graphs\DATA\Experiments\R-C3H3F3O_550eV_CR_newEfield_ALLCH_MFPAD_30_t0.root")
fileSCR = uproot.open(r"C:\Users\Giammarco\Desktop\PYTHON_graphs\DATA\Experiments\S-C3H3F3O_550eV_CR_newEfield_ALLCH_MFPAD_30_t0.root")
fileRCL = uproot.open(r"C:\Users\Giammarco\Desktop\PYTHON_graphs\DATA\Experiments\R-C3H3F3O_550eV_CL_newEfield_ALLCH_MFPAD_30_t0.root")
fileSCL = uproot.open(r"C:\Users\Giammarco\Desktop\PYTHON_graphs\DATA\Experiments\S-C3H3F3O_550eV_CL_newEfield_ALLCH_MFPAD_30_t0.root")

In [ ]:
def smoothgauss(MFPAD, sigmax, sigmay):
    """
    Fucntion the smooths with a 2D gaussian function 2D matrixes. For the experiments
    x = cos(theta), y = phi. Phi is twice ctheta and sigma should be the same.
    """
    # NOTE: the right order is y,x (from stackoverflow)
    sigma = [sigmay,sigmax] 
    Y = sp.ndimage.filters.gaussian_filter(MFPAD, sigma, mode="constant")
    return Y

In [ ]:
histRCR = fileRCR[path]
histSCR = fileSCR[path]
histRCL = fileRCL[path]
histSCL = fileSCL[path]

In [ ]:
phistRCR = histRCR.pandas()
nhistRCR = histRCR.numpy()
r_norm_nhistRCR = nhistRCR[0] / nhistRCR[0].sum(axis=1)[:,np.newaxis] # r values normalise to 1
phi_nhistRCR = nhistRCR[1][0][0] # phi
costheta_nhistRCR = nhistRCR[1][0][1] # cos(theta)

phistRCL = histRCL.pandas()
nhistRCL = histRCL.numpy()
r_norm_nhistRCL = nhistRCL[0] / nhistRCL[0].sum(axis=1)[:,np.newaxis] # z values normalise to 1
phi_nhistRCL = nhistRCL[1][0][0] # phi
costheta_nhistRCL = nhistRCL[1][0][1] # cos(theta)

phistSCR = histSCR.pandas()
nhistSCR = histSCR.numpy()
r_norm_nhistSCR = nhistSCR[0] / nhistSCR[0].sum(axis=1)[:,np.newaxis] # z values normalise to 1
phi_nhistSCR = nhistSCR[1][0][0] # phi
costheta_nhistSCR = nhistSCR[1][0][1] # cos(theta)

phistSCL = histSCL.pandas()
nhistSCL = histSCL.numpy()
r_norm_nhistSCL = nhistSCL[0] / nhistSCL[0].sum(axis=1)[:,np.newaxis] # z values normalise to 1
phi_nhistSCL = nhistSCL[1][0][0] # phi
costheta_nhistSCL = nhistSCL[1][0][1] # cos(theta)

totR = np.add(r_norm_nhistRCL,r_norm_nhistRCR)
totS = np.add(r_norm_nhistSCL,r_norm_nhistSCR)

totR_sm = smoothgauss(totR,0.3,0.6)
totS_sm = smoothgauss(totS,0.3,0.6)

PECD =  np.divide(np.subtract(totR,totS),np.add(totR,totS))
PECD_smooth = np.divide(np.subtract(totR_sm,totS_sm),np.add(totR_sm,totS_sm))

cosM = (costheta_nhistSCL[1:] + costheta_nhistSCL[:-1])/2
phiM = (phi_nhistSCL[1:] + phi_nhistSCL[:-1])/2

In [ ]:
nhistRCL[0].shape

In [ ]:
(nhistRCL[0]/nhistRCL[0].sum(axis=1)[:,np.newaxis]).shape

In [ ]:
print(r_norm_nhistRCR.shape)
print(nhistRCR[0].shape)
print(phi_nhistRCR.shape)
print(costheta_nhistRCR.shape)
# x=r_norm_nhistRCL*np.sin(np.arccos(costheta_nhistRCL))*np.cos(phi_nhistRCL)
# y=r_norm_nhistRCL*np.sin(np.arccos(costheta_nhistRCL))*np.sin(phi_nhistRCL)
# z=r_norm_nhistRCL*costheta_nhistRCL

In [ ]:
clev = np.arange(PECD.min(),PECD.max(),.01)
clev_mod = np.arange(-0.15,0.15,.01)

cs = plt.contourf(phiM, cosM, PECD.T, 10, extent="both", aspect="auto", cmap='seismic') # first option with increasing number of levels = 10
cbar = plt.colorbar(cs)
plt.title("Contrast map 10 levels")
plt.show()

#better appearance, but probaly not accurate because runs till the end of the scale
# extent= [phi_nhistSCL[0],phi_nhistSCL[-1],costheta_nhistSCL[0],costheta_nhistSCL[-1]]

cs1 = plt.contourf(PECD.T, clev, extent= [phi_nhistSCL[0],phi_nhistSCL[-1],costheta_nhistSCL[0],costheta_nhistSCL[-1]], aspect="equal",cmap='seismic') # first option with increasing number of levels
cbar = plt.colorbar(cs1)
plt.title("Contrast map thin levels")
plt.show()

cs2 = plt.contourf(PECD_smooth.T, clev, extent= [phi_nhistSCL[0],phi_nhistSCL[-1],costheta_nhistSCL[0],costheta_nhistSCL[-1]], aspect="equal",cmap='seismic') # first option with increasing number of levels
cbar = plt.colorbar(cs2)
plt.title("Contrast map thin levels smoothed")
plt.show()

In [ ]:
# interpolation: [ 'none', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos'.]

fig=plt.figure()
plt.imshow(PECD.T, vmin=-0.15, vmax=0.15, extent= [phi_nhistSCL[0],phi_nhistSCL[-1],costheta_nhistSCL[0],costheta_nhistSCL[-1]], interpolation='quadric', cmap='seismic', aspect="auto")
plt.title("Contrast map quadratic interpolation")
plt.colorbar()
# plt.axis(aspect='image');

fig1=plt.figure()
plt.imshow(PECD_smooth.T, vmin=-0.10, vmax=0.10, extent= [phi_nhistSCL[0],phi_nhistSCL[-1],costheta_nhistSCL[0],costheta_nhistSCL[-1]], interpolation='quadric', cmap='seismic', aspect="auto")
# interpolation: [bicubic, quadric, spline16]
plt.title("Contrast map smoothed quadratic interpolation")

#modifies just the colobar
# plt.colorbar(boundaries=np.linspace(-.05, .05, 5, endpoint=True))
# plt.axis(aspect='image');
plt.colorbar()

In [ ]:
import matplotlib as mpl
from matplotlib import cm

magma_cmap = mpl.cm.get_cmap('magma')
seismic_cmap = mpl.cm.get_cmap('seismic')

magma_rgb = []
seismic_rgb = []
norm = mpl.colors.Normalize(vmin=0, vmax=255)

for i in range(0, 255):
       k = mpl.colors.colorConverter.to_rgb(magma_cmap(norm(i)))
       magma_rgb.append(k)

for i in range(0, 255):
       k = mpl.colors.colorConverter.to_rgb(seismic_cmap(norm(i)))
       seismic_rgb.append(k)

def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

Magma_r = matplotlib_to_plotly(magma_cmap, 255)
Seismic_r = matplotlib_to_plotly(seismic_cmap, 255)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = go.Figure(data=go.Contour(x=phiM, y=cosM, z=PECD_smooth.T, line_smoothing=0.85, colorscale=Seismic_r,
contours=dict(start=-0.12, end=0.12, size=0.015) 
))

fig.update_layout(
    # coloraxis=dict(colorscale=Seismic_r),
    # showlegend=False,
    autosize=False,
    width=500,
    height=450,
    margin=dict(l=5,r=5,b=25,t=25)
    )

fig.show()